**Table of contents**<a id='toc0_'></a>    
- [Data Preprocessing: Insomniac Games Attack](#toc1_)    
  - [Loading Posts, Comments, and Replies](#toc1_1_)    
  - [Removing duplicates](#toc1_2_)    
  - [Removing based on date of post](#toc1_3_)    
  - [Removing off topic entries](#toc1_4_)    
- [Data Preprocessing: WannaCry Attack](#toc2_)    
  - [Load](#toc2_1_)    
  - [Remove duplicates](#toc2_2_)    
  - [Remove by date](#toc2_3_)    
  - [Remove off topic entries](#toc2_4_)    
- [Data Preprocessing: SolarWinds Attack](#toc3_)    
  - [Load](#toc3_1_)    
  - [Remove duplicates](#toc3_2_)    
  - [Remove by date](#toc3_3_)    
  - [Remove off topic entries](#toc3_4_)    
- [Merge datasets and final steps](#toc4_)    
  - [Store the results for usage](#toc4_1_)    
  - [Removing Non-English content](#toc4_2_)    
  - [Text Cleaning](#toc4_3_)    
  - [Spelling Correction](#toc4_4_)    
  - [Lemmatization and stopword removal](#toc4_5_)    
- [Save pre-processed data](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import re
import warnings
import nltk
import sqlite3
import numpy as np
import pandas as pd
from typing import Tuple
from transformers import pipeline
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from autocorrect import Speller
from langdetect import detect

In [2]:
# Filter out warnings, they were annoying 
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'

# <a id='toc1_'></a>[Data Preprocessing: Insomniac Games Attack](#toc0_)

## <a id='toc1_1_'></a>[Loading Posts, Comments, and Replies](#toc0_)

In [3]:
conn = sqlite3.connect('Collected.db')
attacks = pd.read_sql(sql = "select * from attacks", con = conn, index_col = "attack_id" )
attacks

,attack_name
attack_id,
1,Insomniac Games
2,WannaCry
3,SolarWinds


In [4]:
def fetch_attack(attack_id: int) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]: 
    posts = pd.read_sql(sql = f"select * from reddit_posts where attack_id = {attack_id}", con = conn, index_col = "rp_index")
    comments = pd.read_sql(sql = f"select * from reddit_parent_comments where post_id in" 
                           f"(select post_id from reddit_posts where attack_id = {attack_id})", con = conn, index_col = "rc_index")
    replies = pd.read_sql(sql = f"select * from reddit_replies where parent_id in" 
                          f"(select parent_comment_id from reddit_parent_comments where post_id in" 
                          f"(select post_id from reddit_posts where attack_id = {attack_id}))",con = conn, index_col = "rr_index")
    return posts, comments, replies

In [5]:
insomniac_posts, insomniac_parent_comments, insomniac_reddit_replies = fetch_attack(1) 

In [6]:
insomniac_posts

,post_id,subreddit,title,content,score,upvote_ratio,post_datetime,attack_id
rp_index,,,,,,,,
1,18mf8b5,TwoBestFriendsPlay,Statement from Remedy on the Insomniac Games r...,,505,0.98,1703027019,1
2,18mf8b5,TwoBestFriendsPlay,Statement from Remedy on the Insomniac Games r...,,497,0.98,1703027019,1
3,18ogglo,pcgaming,Insomniac Games Releases Statement on recent r...,,630,0.90,1703256167,1
4,18gkw38,SpidermanPS4,"So apparently, Insomniac got hit with a ransom...",,1584,0.88,1702384490,1
5,18lzgnk,Marvel,Several Insomniac Marvel games and estimate re...,,208,0.88,1702984282,1
...,...,...,...,...,...,...,...,...
2355,18vx15o,videogames,Which games do you like more - The Batman Arkh...,,459,0.92,1704116661,1
2356,10ru6wt,Games,Rumor: Insomniac’s Wolverine game is reportedl...,,6284,0.96,1675355840,1
2357,1byqmuy,InsomniacGames,How would you rank these 6 Insomniac games?,,668,1.00,1712556487,1


In [7]:
insomniac_parent_comments

,parent_comment_id,content,score,comment_datetime,post_id
rc_index,,,,,
1,ke3saoh,The personal info getting leaked is honestly s...,232,1703027544,18mf8b5
2,ke3u1iu,It's one thing to leak the games for everyone ...,136,1703028248,18mf8b5
3,ke45kdu,It's a sobering reminder that the people who d...,73,1703032967,18mf8b5
4,ke3xow5,Although this may strike some readers as harsh...,75,1703029736,18mf8b5
5,ke3xbdr,It's good that Insom is at least receiving sup...,51,1703029581,18mf8b5
...,...,...,...,...,...
227748,j5pbeng,>\tFrom developing the best VR headset on the ...,3,1674579887,10k7l1p
227749,j5pk2kj,"For those reading this thread in the future, w...",4,1674583040,10k7l1p
227750,j5pywow,> Was it lackluster for having waited over a d...,3,1674588451,10k7l1p


In [8]:
insomniac_reddit_replies

,reply_id,content,score,comment_datetime,parent_id,post_id
rr_index,,,,,,
1,ke3sw9d,Right? I'll have *no* sympathy for the hackers...,162,1703027787,ke3saoh,18mf8b5
2,ke4grbb,"It sucks for a couple reasons\n\n1- timing, it...",57,1703037616,ke3saoh,18mf8b5
3,ke496do,Especially since a certain crowd has already m...,47,1703034454,ke3saoh,18mf8b5
4,ke4tx0q,"Honestly, I wouldn’t get your hopes up for jus...",48,1703043362,ke3saoh,18mf8b5
5,ke47jzq,[removed],-44,1703033783,ke3saoh,18mf8b5
...,...,...,...,...,...,...
654931,j5pbeng,>\tFrom developing the best VR headset on the ...,3,1674579887,j5pasxl,10k7l1p
654932,j5pk2kj,"For those reading this thread in the future, w...",4,1674583040,j5phwp0,10k7l1p
654933,j5pywow,> Was it lackluster for having waited over a d...,3,1674588451,j5phwp0,10k7l1p


In [9]:
print(np.shape(insomniac_posts))
print(np.shape(insomniac_parent_comments))
print(np.shape(insomniac_reddit_replies))

(363, 8)
(51916, 5)
(134466, 6)


## <a id='toc1_2_'></a>[Removing duplicates](#toc0_)


In [10]:
insomniac_posts = insomniac_posts.drop_duplicates(subset = 'post_id')
insomniac_parent_comments = insomniac_parent_comments.drop_duplicates(subset = 'parent_comment_id')
insomniac_reddit_replies = insomniac_reddit_replies.drop_duplicates(subset = 'reply_id')
print(np.shape(insomniac_posts))
print(np.shape(insomniac_parent_comments))
print(np.shape(insomniac_reddit_replies))

(238, 8)
(35391, 5)
(25215, 6)


## <a id='toc1_3_'></a>[Removing based on date of post](#toc0_)
- Remove all posts earlier than 2 weeks before the attack beginning.
- Each attack has a date that can be pinpointed as the start of the attack. 
- In the case of solarwinds, this will be treated as the date of its discovery

https://news.sky.com/story/wolverine-what-we-know-about-the-cyberattack-that-leaked-one-of-playstations-most-anticipated-games-13034721#:~:text=When%20and%20how%20did%20the,stolen%20data%20to%20be%20leaked.

In [11]:
def remove_by_date(cutoff_date: int, posts: pd.DataFrame, parent_comments: pd.DataFrame, reddit_replies: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    # Filter posts based on the cutoff date
    posts = posts[posts['post_datetime'] >= cutoff_date]
    # Filter parent_comments based on the cutoff date
    parent_comments = parent_comments[parent_comments['comment_datetime'] >= cutoff_date]
    # Filter reddit_replies based on the cutoff date
    reddit_replies = reddit_replies[reddit_replies['comment_datetime'] >= cutoff_date]
    return posts, parent_comments, reddit_replies

In [12]:
insomniac_posts, insomniac_parent_comments, insomniac_reddit_replies = remove_by_date(1701129600, insomniac_posts, insomniac_parent_comments, insomniac_reddit_replies)
print(np.shape(insomniac_posts))
print(np.shape(insomniac_parent_comments))
print(np.shape(insomniac_reddit_replies))

(148, 8)
(13021, 5)
(8830, 6)


## <a id='toc1_4_'></a>[Removing off topic entries](#toc0_)

- facebook/bart-large-mnli model perfroms zero-shot classification
- This is topic classification, each title will get a score based on its likelihood of being about each given topic. 
- Posts below the minimum cut-off point are removed

In [13]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [14]:
def get_off_topic(titles: list, labels: list) -> list:
    classifier_outputs = []
    for title in titles:
        sequence_to_classify=title
        classifier_outputs.append(classifier(sequence_to_classify, labels, multi_label = True))
    off_topic = []
    for index, output in enumerate(classifier_outputs):
        if max(output['scores']) <= 0.65: # Scores cut off point.
            off_topic.append({'index': index, 'title': output['sequence']})
    return off_topic    

In [15]:
insomniac_titles = insomniac_posts['title']
insomniac_titles   

rp_index
1       Statement from Remedy on the Insomniac Games r...
3       Insomniac Games Releases Statement on recent r...
4       So apparently, Insomniac got hit with a ransom...
5       Several Insomniac Marvel games and estimate re...
6       Insomniac Games alerts employees hit by ransom...
                              ...                        
2342    Who's your favorite and least favorite charact...
2347      Insomniac leak: Demand for the remastered games
2354    Should insomniac make a spider man 2099 game (...
2355    Which games do you like more - The Batman Arkh...
2357          How would you rank these 6 Insomniac games?
Name: title, Length: 148, dtype: object

In [16]:
insomniac_labels = ["Insomniac leak", "Wolverine leak", "Spider-Man leak", "Insomniac cyber attack", "Insomniac ransomware", "Insomniac hack", "Insomniac stolen information"]
insomniac_labels

['Insomniac leak',
 'Wolverine leak',
 'Spider-Man leak',
 'Insomniac cyber attack',
 'Insomniac ransomware',
 'Insomniac hack',
 'Insomniac stolen information']

In [17]:
insomniac_off_topic =  get_off_topic(insomniac_titles, insomniac_labels)

In [18]:
insomniac_off_topic

[{'index': 29,
  'title': 'Rhysida užpuolė "Sony" dukterinę įmonę "Insomniac Games"'},
 {'index': 32,
  'title': 'So... should gaming journalists be gaming journalists?'},
 {'index': 39,
  'title': 'Discussion, News, and Request Thread - week beginning 12/17/23'},
 {'index': 43, 'title': 'Epic hackerata'},
 {'index': 46, 'title': 'Cyber Briefing - 2023.12.12'},
 {'index': 47, 'title': 'news del 12.12.23'},
 {'index': 48, 'title': 'Monthly data breach alert'},
 {'index': 49,
  'title': 'Week 51 | Cybersecurity - technology - privacy News recap'},
 {'index': 55,
  'title': 'DQM: The Dark Prince Accessories List with effects and which monster drops them'},
 {'index': 59,
  'title': 'Sysadmins that work for Game Studios: is it as fucking awful as it seems?'},
 {'index': 60,
  'title': 'Do breaches actually cause for a loss in trust and credibility?'},
 {'index': 61, 'title': 'i got sued, been a fun time boys'},
 {'index': 62,
  'title': 'LittleBigPlanet 3 has sold 5.4mil units (making it t

In [19]:
def remove_off_topic(off_topic: list, posts: pd.DataFrame, parent_comments: pd.DataFrame, reddit_replies: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    post_ids = []
    # Iterate through each title and remove accordingly
    for i in off_topic:
        posts_to_remove = posts[posts['title'] == i['title']]
        post_ids.extend(np.unique([posts_to_remove["post_id"]]))
    posts = posts[~posts['post_id'].isin(post_ids)]
    parent_comments = parent_comments[~parent_comments['post_id'].isin(post_ids)]
    reddit_replies = reddit_replies[~reddit_replies['post_id'].isin(post_ids)]
    return posts, parent_comments, reddit_replies

In [20]:
insomniac_posts, insomniac_parent_comments, insomniac_reddit_replies = remove_off_topic(insomniac_off_topic, insomniac_posts, insomniac_parent_comments, insomniac_reddit_replies)
print(np.shape(insomniac_posts))
print(np.shape(insomniac_parent_comments))
print(np.shape(insomniac_reddit_replies))

(96, 8)
(8107, 5)
(6394, 6)


# <a id='toc2_'></a>[Data Preprocessing: WannaCry Attack](#toc0_)

## <a id='toc2_1_'></a>[Load](#toc0_)

In [21]:
wannacry_posts, wannacry_parent_comments, wannacry_reddit_replies = fetch_attack(2)

In [22]:
wannacry_posts

,post_id,subreddit,title,content,score,upvote_ratio,post_datetime,attack_id
rp_index,,,,,,,,
314,6cmmdf,IAmA,"IamA the ""accidental hero"" who helped stop the...","**My short bio:** Hey I'm MalwareTech, a malwa...",24035,0.88,1495451693,2
315,6b7gd4,news,Microsoft president blasts NSA for its role in...,,26998,0.91,1494810543,2
316,6bs66v,dataisbeautiful,Balance of the bitcoin wallets tied to the Wan...,,4428,0.94,1495060723,2
317,nwa7t4,cybersecurity,A WannaCry documentary that I made,"Hi everyone,\n\nnot sure if I'm allowed to be ...",734,0.99,1623283908,2
318,gid4jk,hacking,"Confessions of Marcus Hutchins, the hacker who...",,664,0.98,1589296605,2
...,...,...,...,...,...,...,...,...
2441,6aujob,technology,Ransomeware Cyberattack Mega-Thread,"Hi folks, \n\nIn light of the ongoing world-wi...",1002,0.93,1494629211,2
2442,1byxk37,conspiracy,WannaCry Ransomware Attack Was Launched by N.S...,\n\nhttps://preview.redd.it/h4drfasm99tc1.png...,0,0.48,1712582043,2
2443,hbpc9o,worldnews,Australia hit by massive cyber attack,,31975,0.93,1592522346,2


In [23]:
wannacry_parent_comments

,parent_comment_id,content,score,comment_datetime,post_id
rc_index,,,,,
14980,dhvr48z,First of all - you have the thanks of many! I'...,3420,1495451975,6cmmdf
14981,dhvrx7q,Did you receive some job offerings from govern...,2220,1495453860,6cmmdf
14982,dhvr4pb,"What's your PC setup specs?\n\nAlso, what VM s...",2178,1495452007,6cmmdf
14983,dhvr463,What are some good resources or ways to learn ...,1336,1495451969,6cmmdf
14984,dhvs8md,"HELLO SIR, GOOD WORK WITH THE KILL SWITCH. MY ...",1327,1495454550,6cmmdf
...,...,...,...,...,...
230686,dhwys1y,"Hey trey, long time fan of the company, first ...",5,1495508162,6cmmdf
230687,dhwwjhq,The best thing you can do is read the original...,3,1495505343,6cmmdf
230688,dhx2r1u,"Hey Josh, glad you chose Emoji-propriate Inc. ...",7,1495514143,6cmmdf


In [24]:
wannacry_reddit_replies

,reply_id,content,score,comment_datetime,parent_id,post_id
rr_index,,,,,,
39789,dhvs1up,I've always wanted to do educational videos an...,4709,1495454139,dhvr48z,6cmmdf
39790,dhvur2r,Is this the beginning of a YouTuber?,1301,1495459154,dhvr48z,6cmmdf
39791,dhw4ay3,You can probably be a keynote speaker for any ...,28,1495471292,dhvr48z,6cmmdf
39792,dhw8piv,"Man, you should get in contact with Brady Hara...",13,1495476307,dhvr48z,6cmmdf
39793,dhw4u90,A TED talk would be awesome,10,1495471900,dhvr48z,6cmmdf
...,...,...,...,...,...,...
663396,dhwwys4,> my job revolves around googling\n\nThis is t...,3,1495505880,dhwsue8,6cmmdf
663397,dhx2r1u,"Hey Josh, glad you chose Emoji-propriate Inc. ...",7,1495514143,dhwys1y,6cmmdf
663398,dhwwwtx,"Thanks a bunch. And no worries, my job revolve...",3,1495505815,dhwwjhq,6cmmdf


## <a id='toc2_2_'></a>[Remove duplicates](#toc0_)

In [25]:
wannacry_posts = wannacry_posts.drop_duplicates(subset = 'post_id')
wannacry_posts

,post_id,subreddit,title,content,score,upvote_ratio,post_datetime,attack_id
rp_index,,,,,,,,
314,6cmmdf,IAmA,"IamA the ""accidental hero"" who helped stop the...","**My short bio:** Hey I'm MalwareTech, a malwa...",24035,0.88,1495451693,2
315,6b7gd4,news,Microsoft president blasts NSA for its role in...,,26998,0.91,1494810543,2
316,6bs66v,dataisbeautiful,Balance of the bitcoin wallets tied to the Wan...,,4428,0.94,1495060723,2
317,nwa7t4,cybersecurity,A WannaCry documentary that I made,"Hi everyone,\n\nnot sure if I'm allowed to be ...",734,0.99,1623283908,2
318,gid4jk,hacking,"Confessions of Marcus Hutchins, the hacker who...",,664,0.98,1589296605,2
...,...,...,...,...,...,...,...,...
2375,1ax59d1,wallstreetbets,Cyber attack? AT&T is down premarket,,1122,0.92,1708604948,2
2378,7kpiac,politics,US to announce North Korea behind WannaCry cyb...,,38,0.82,1513643331,2
2379,lfxs44,gaming,CD projekt red just got cyber attacked,,45662,0.95,1612857917,2


In [26]:
wannacry_parent_comments = wannacry_parent_comments.drop_duplicates(subset ='parent_comment_id')
wannacry_parent_comments

,parent_comment_id,content,score,comment_datetime,post_id
rc_index,,,,,
14980,dhvr48z,First of all - you have the thanks of many! I'...,3420,1495451975,6cmmdf
14981,dhvrx7q,Did you receive some job offerings from govern...,2220,1495453860,6cmmdf
14982,dhvr4pb,"What's your PC setup specs?\n\nAlso, what VM s...",2178,1495452007,6cmmdf
14983,dhvr463,What are some good resources or ways to learn ...,1336,1495451969,6cmmdf
14984,dhvs8md,"HELLO SIR, GOOD WORK WITH THE KILL SWITCH. MY ...",1327,1495454550,6cmmdf
...,...,...,...,...,...
210158,drhc468,let's think about it.... a poor country with b...,1,1513706744,7kphs3
210159,drhd0ss,"The war sell itself, fuck nuclear proliferatio...",1,1513707674,7kphs3
210160,drhdi6v,War never sells itself unless we’re attacked d...,2,1513708153,7kphs3


In [27]:
wannacry_reddit_replies = wannacry_reddit_replies.drop_duplicates(subset = 'reply_id')
wannacry_reddit_replies

,reply_id,content,score,comment_datetime,parent_id,post_id
rr_index,,,,,,
39789,dhvs1up,I've always wanted to do educational videos an...,4709,1495454139,dhvr48z,6cmmdf
39790,dhvur2r,Is this the beginning of a YouTuber?,1301,1495459154,dhvr48z,6cmmdf
39791,dhw4ay3,You can probably be a keynote speaker for any ...,28,1495471292,dhvr48z,6cmmdf
39792,dhw8piv,"Man, you should get in contact with Brady Hara...",13,1495476307,dhvr48z,6cmmdf
39793,dhw4u90,A TED talk would be awesome,10,1495471900,dhvr48z,6cmmdf
...,...,...,...,...,...,...
594143,drgw9zm,It's easy to spoof IP address or sprinkle Russ...,3,1513687945,drgitsw,7kphs3
594144,drgjsfz,"I don't know about those stuffs,but according ...",-2,1513660701,drgitsw,7kphs3
594145,drgz7zx,Yeah but in that case they'd maybe not use day...,1,1513692603,drgitsw,7kphs3


## <a id='toc2_3_'></a>[Remove by date](#toc0_)

https://www.vox.com/new-money/2017/5/15/15641196/wannacry-ransomware-windows-xp


In [28]:
wannacry_posts, wannacry_parent_comments, wannacry_reddit_replies = remove_by_date(1493161200, wannacry_posts, wannacry_parent_comments, wannacry_reddit_replies)
print(np.shape(wannacry_posts))
print(np.shape(wannacry_parent_comments))
print(np.shape(wannacry_reddit_replies))

(811, 8)
(66752, 5)
(45644, 6)


## <a id='toc2_4_'></a>[Remove off topic entries](#toc0_)

In [29]:
wannacry_titles = wannacry_posts['title']
wannacry_titles   

rp_index
314     IamA the "accidental hero" who helped stop the...
315     Microsoft president blasts NSA for its role in...
316     Balance of the bitcoin wallets tied to the Wan...
317                    A WannaCry documentary that I made
318     Confessions of Marcus Hutchins, the hacker who...
                              ...                        
2375                Cyber attack? AT&T is down premarket 
2378    US to announce North Korea behind WannaCry cyb...
2379               CD projekt red just got cyber attacked
2380    U.S. to blame North Korea for 'WannaCry' cyber...
2381    Finland Hit by Cyber Attack, Airspace Breach a...
Name: title, Length: 811, dtype: object

In [30]:
wannacry_labels = ["WannaCry attack", "WannaCry cyber attack", "WannaCry hack", "WannaCry ransomware", "WannaCry Windows", "WannaCry Bitcoin"]
wannacry_labels

['WannaCry attack',
 'WannaCry cyber attack',
 'WannaCry hack',
 'WannaCry ransomware',
 'WannaCry Windows',
 'WannaCry Bitcoin']

In [31]:
wannacry_off_topic = get_off_topic(wannacry_titles, wannacry_labels)

In [32]:
wannacry_posts, wannacry_parent_comments, wannacry_reddit_replies = remove_off_topic(wannacry_off_topic, wannacry_posts, wannacry_parent_comments, wannacry_reddit_replies)
print(np.shape(wannacry_posts))
print(np.shape(wannacry_parent_comments))
print(np.shape(wannacry_reddit_replies))

(419, 8)
(17355, 5)
(14569, 6)


# <a id='toc3_'></a>[Data Preprocessing: SolarWinds Attack](#toc0_)

## <a id='toc3_1_'></a>[Load](#toc0_)

In [33]:
solarwinds_posts, solarwinds_parent_comments, solarwinds_reddit_replies = fetch_attack(3)

## <a id='toc3_2_'></a>[Remove duplicates](#toc0_)

In [34]:
solarwinds_posts = solarwinds_posts.drop_duplicates(subset ='post_id')
solarwinds_posts

,post_id,subreddit,title,content,score,upvote_ratio,post_datetime,attack_id
rp_index,,,,,,,,
1360,lk37qq,worldnews,SolarWinds hack was 'largest and most sophisti...,,14737,0.97,1613355014,3
1361,lkt35u,news,"Microsoft says it found 1,000-plus developers'...",,4221,0.97,1613440951,3
1362,lk9wu2,programming,"Microsoft says it found 1,000-plus developers'...",,1824,0.92,1613380225,3
1363,kgwi6h,worldnews,Biden is considering Russian financial sanctio...,,38014,0.90,1608480284,3
1364,kdx0p9,technology,No One Knows How Deep Russia's Hacking Rampage...,,11515,0.96,1608074277,3
...,...,...,...,...,...,...,...,...
2305,mg48j9,RedPacketSecurity,SolarWinds Patches Four New Vulnerabilities in...,,1,1.00,1617066068,3
2307,hb1zu9,msp,SolarWinds RMM: Security Notice Regarding An A...,Just an FYI for those using Solarwinds RMM\n\n...,28,0.84,1592433844,3
2308,ld4f7i,ScienceUncensored,SolarWinds patches vulnerabilities that could ...,,2,1.00,1612520829,3


In [35]:
solarwinds_parent_comments = solarwinds_parent_comments.drop_duplicates(subset = 'parent_comment_id')
solarwinds_parent_comments

,parent_comment_id,content,score,comment_datetime,post_id
rc_index,,,,,
27238,h4tmgo8,God needs to nerf Supply Chain attacks immedia...,46,1626021381,oi6wwa
27239,h4tnqbp,"Great video, you've got a lot of potential! Ex...",17,1626022023,oi6wwa
27240,h4tqcgt,"Loved your WANNACRY video, excited to see this...",9,1626023330,oi6wwa
27241,h4tvwqa,Awesome video,4,1626026103,oi6wwa
27242,h4txnrp,I have literally been looking for a solar wind...,3,1626026975,oi6wwa
...,...,...,...,...,...
220411,gsow4wk,Shocker. Well. Not really a shocker.,6,1617028497,mfhuq7
220412,gsq4dj7,But Bush/Obama did! They started this war with...,1,1617048746,mfhuq7
220413,gspo05b,That's true. We're all blessed that he was inc...,1,1617041266,mfhuq7


In [36]:
solarwinds_reddit_replies = solarwinds_reddit_replies.drop_duplicates(subset ='reply_id')
solarwinds_reddit_replies

,reply_id,content,score,comment_datetime,parent_id,post_id
rr_index,,,,,,
78198,h4tmpkk,"I second this 😂 as it stands, it's incredibly OP",12,1626021505,h4tmgo8,oi6wwa
78199,h4tny9g,Thank you so much!,6,1626022133,h4tnqbp,oi6wwa
78200,h4tqwav,Thank you so much :') looking forward for some...,3,1626023609,h4tqcgt,oi6wwa
78201,h4y4hkj,"It was great, informative and entertaining. I ...",2,1626113807,h4tqcgt,oi6wwa
78202,h50sgsg,Oh I'm sorry to hear that :/ I'll amp it up ne...,1,1626171366,h4tqcgt,oi6wwa
...,...,...,...,...,...,...
629989,gsow4wk,Shocker. Well. Not really a shocker.,6,1617028497,gsorx9x,mfhuq7
629990,gsq4dj7,But Bush/Obama did! They started this war with...,1,1617048746,gsorx9x,mfhuq7
629991,gspo05b,That's true. We're all blessed that he was inc...,1,1617041266,gsorx9x,mfhuq7


## <a id='toc3_3_'></a>[Remove by date](#toc0_)

https://www.csoonline.com/article/570537/the-solarwinds-hack-timeline-who-knew-what-and-when.html

In [37]:
solarwinds_posts, solarwinds_parent_comments, solarwinds_reddit_replies = remove_by_date(1606176000, solarwinds_posts, solarwinds_parent_comments, solarwinds_reddit_replies)
print(np.shape(solarwinds_posts))
print(np.shape(solarwinds_parent_comments))
print(np.shape(solarwinds_reddit_replies))

(636, 8)
(24960, 5)
(19055, 6)


## <a id='toc3_4_'></a>[Remove off topic entries](#toc0_)

In [38]:
solarwinds_titles = solarwinds_posts['title']
solarwinds_titles   

rp_index
1360    SolarWinds hack was 'largest and most sophisti...
1361    Microsoft says it found 1,000-plus developers'...
1362    Microsoft says it found 1,000-plus developers'...
1363    Biden is considering Russian financial sanctio...
1364    No One Knows How Deep Russia's Hacking Rampage...
                              ...                        
2304    SolarWinds patches vulnerabilities that could ...
2305    SolarWinds Patches Four New Vulnerabilities in...
2308    SolarWinds patches vulnerabilities that could ...
2309    A deep-dive into the SolarWinds Serv-U SSH vul...
2431    AP sources: SolarWinds hack got emails of top ...
Name: title, Length: 636, dtype: object

In [39]:
solarwinds_labels = ["SolarWinds attack", "SolarWinds cyber attack", "SolarWinds hack", "SolarWinds exploit", "SolarWinds data breach"]
solarwinds_labels

['SolarWinds attack',
 'SolarWinds cyber attack',
 'SolarWinds hack',
 'SolarWinds exploit',
 'SolarWinds data breach']

In [40]:
solarwinds_off_topic = get_off_topic(solarwinds_titles, solarwinds_labels)

In [41]:
solarwinds_posts, solarwinds_parent_comments, solarwinds_reddit_replies = remove_off_topic(solarwinds_off_topic, solarwinds_posts, solarwinds_parent_comments, solarwinds_reddit_replies)
print(np.shape(solarwinds_posts))
print(np.shape(solarwinds_parent_comments))
print(np.shape(solarwinds_reddit_replies))

(360, 8)
(16640, 5)
(13415, 6)


# <a id='toc4_'></a>[Merge datasets and final steps](#toc0_)

- Next processes are not specific to one attack so merging justifiable


In [42]:
def merge_datasets(insomniac_df: pd.DataFrame, wannacry_df: pd.DataFrame, solarwinds_df: pd.DataFrame) -> pd.DataFrame:
    merged_df = pd.concat(objs = [insomniac_df, wannacry_df, solarwinds_df])
    return merged_df

In [43]:
merged_posts = merge_datasets(insomniac_posts, wannacry_posts, solarwinds_posts) 
merged_posts

,post_id,subreddit,title,content,score,upvote_ratio,post_datetime,attack_id
rp_index,,,,,,,,
1,18mf8b5,TwoBestFriendsPlay,Statement from Remedy on the Insomniac Games r...,,505,0.98,1703027019,1
3,18ogglo,pcgaming,Insomniac Games Releases Statement on recent r...,,630,0.90,1703256167,1
4,18gkw38,SpidermanPS4,"So apparently, Insomniac got hit with a ransom...",,1584,0.88,1702384490,1
5,18lzgnk,Marvel,Several Insomniac Marvel games and estimate re...,,208,0.88,1702984282,1
6,1b0m4vj,technology,Insomniac Games alerts employees hit by ransom...,,62,0.88,1708965991,1
...,...,...,...,...,...,...,...,...
2304,lcf39o,technology,SolarWinds patches vulnerabilities that could ...,,4,0.99,1612443514,3
2305,mg48j9,RedPacketSecurity,SolarWinds Patches Four New Vulnerabilities in...,,1,1.00,1617066068,3
2308,ld4f7i,ScienceUncensored,SolarWinds patches vulnerabilities that could ...,,2,1.00,1612520829,3


In [44]:
merged_parent_comments = merge_datasets(insomniac_parent_comments, wannacry_parent_comments, solarwinds_parent_comments) 
merged_parent_comments

,parent_comment_id,content,score,comment_datetime,post_id
rc_index,,,,,
1,ke3saoh,The personal info getting leaked is honestly s...,232,1703027544,18mf8b5
2,ke3u1iu,It's one thing to leak the games for everyone ...,136,1703028248,18mf8b5
3,ke45kdu,It's a sobering reminder that the people who d...,73,1703032967,18mf8b5
4,ke3xow5,Although this may strike some readers as harsh...,75,1703029736,18mf8b5
5,ke3xbdr,It's good that Insom is at least receiving sup...,51,1703029581,18mf8b5
...,...,...,...,...,...
220411,gsow4wk,Shocker. Well. Not really a shocker.,6,1617028497,mfhuq7
220412,gsq4dj7,But Bush/Obama did! They started this war with...,1,1617048746,mfhuq7
220413,gspo05b,That's true. We're all blessed that he was inc...,1,1617041266,mfhuq7


In [45]:
merged_reddit_replies = merge_datasets(insomniac_reddit_replies, wannacry_reddit_replies, solarwinds_reddit_replies) 
merged_reddit_replies

,reply_id,content,score,comment_datetime,parent_id,post_id
rr_index,,,,,,
1,ke3sw9d,Right? I'll have *no* sympathy for the hackers...,162,1703027787,ke3saoh,18mf8b5
2,ke4grbb,"It sucks for a couple reasons\n\n1- timing, it...",57,1703037616,ke3saoh,18mf8b5
3,ke496do,Especially since a certain crowd has already m...,47,1703034454,ke3saoh,18mf8b5
4,ke4tx0q,"Honestly, I wouldn’t get your hopes up for jus...",48,1703043362,ke3saoh,18mf8b5
5,ke47jzq,[removed],-44,1703033783,ke3saoh,18mf8b5
...,...,...,...,...,...,...
629989,gsow4wk,Shocker. Well. Not really a shocker.,6,1617028497,gsorx9x,mfhuq7
629990,gsq4dj7,But Bush/Obama did! They started this war with...,1,1617048746,gsorx9x,mfhuq7
629991,gspo05b,That's true. We're all blessed that he was inc...,1,1617041266,gsorx9x,mfhuq7


In [46]:
print(np.shape(merged_posts))
print(np.shape(merged_parent_comments))
print(np.shape(merged_reddit_replies))

(875, 8)
(42102, 5)
(34378, 6)


## <a id='toc4_1_'></a>[Store the results for usage](#toc0_)
- Majority of processing occurred above, this is for speed of access.
- No queries are required here, so csv will do.

In [47]:
merged_posts.to_csv('preprocessing_merged_posts.csv', index=True)  # Save DataFrame to a CSV file without including the index
merged_parent_comments.to_csv('preprocessing_merged_parent_comments.csv', index=True)
merged_reddit_replies.to_csv('preprocessing_merged_reddit_replies.csv', index=True)

In [48]:
merged_posts = pd.read_csv('preprocessing_merged_posts.csv', index_col = 0)  # Save DataFrame to a CSV file without including the index
merged_parent_comments = pd.read_csv('preprocessing_merged_parent_comments.csv', index_col = 0)
merged_reddit_replies = pd.read_csv('preprocessing_merged_reddit_replies.csv', index_col = 0)

## <a id='toc4_2_'></a>[Removing Non-English content](#toc0_)

- The models that are being for sentiment analysis are trained for English language
- Translation is an alternative, but this is flawed (meaning in might get lost/mis-represented) and computationally expensive.
- Only tagged as non-english, will be removed later

In [49]:
def filter_english(df, text_column = "content", posts = False):
    for index, row in df.iterrows():
        text = row[text_column]
        text = str(text)
        try:
            language = detect(text)
            # Handle posts
            if posts == True and language != 'en': 
                row[text_column] = f"language detected: {language} for: {text}"
                # Keep post anyway
            # Check if language is English
            elif language != 'en':  
                row[text_column] = '' 
        except:
            pass  # Skip rows where language detection fails
    return df

In [50]:
merged_posts = filter_english(merged_posts, 'title', posts = True)
merged_parent_comments = filter_english(merged_parent_comments)
merged_reddit_replies = filter_english(merged_reddit_replies)

## <a id='toc4_3_'></a>[Text Cleaning](#toc0_)

- Removal: 
  - Links
  - HTML tags
  - New lines
  - Special characters (emojis and emoticons)
  - Punctuation
  - Extra whitespace

In [51]:
def clean_text(text: str) -> str:
    # Convert text to string
    text = str(text)
    # Deleted/Removed content, content marked for deletion
    if text in ['[removed]', '[deleted]']: 
        text = ''
    # Remove HTML tags
    cleaned_text = re.sub(r'<[^>]+>', '', text)
    # Remove Reddit-specific markup
    cleaned_text = re.sub(r'(\*|_)(.*?)\1', r'\2', cleaned_text)  # Markdown for italics
    cleaned_text = re.sub(r'\*\*(.*?)\*\*', r'\1', cleaned_text)   # Markdown for bold
    cleaned_text = re.sub(r'\[([^]]+)\]\(([^)]+)\)', r'\1', cleaned_text)  # Markdown for hyperlinks
    cleaned_text = re.sub(r'> (.*?)\n', r'\1', cleaned_text)       # Markdown for quoting text
    # Remove new lines
    cleaned_text = cleaned_text.replace('\n', ' ')
    # Remove links
    cleaned_text = re.sub(r'https?://\S+|www\.\S+', '', cleaned_text)
    # Remove Reddit user mentions
    cleaned_text = re.sub(r'/u/\w+', '', cleaned_text)
    # Remove subreddit references
    cleaned_text = re.sub(r'/r/\w+', '', cleaned_text)
    # Remove emojis and emoticons
    # Remove extra whitespace
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text.strip()

In [52]:
merged_parent_comments["content"] = merged_parent_comments["content"].apply(clean_text)
merged_reddit_replies["content"] = merged_reddit_replies["content"].apply(clean_text)

## <a id='toc4_4_'></a>[Spelling Correction](#toc0_)

In [53]:
def chunk_text(text, max_words_per_chunk=20):
    # Split data up into chunks of 20
    words = text.split()
    chunks = []
    current_chunk = []
    for word in words:
        current_chunk.append(word)
        if len(current_chunk) >= max_words_per_chunk:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

spell = Speller(fast = True)
def autocorrect_text(text):
    # Without this setting, it would not complete in a reasonable timeframe 
    # Split the text into smaller chunks
    chunks = chunk_text(text)
    # Process each chunk separately
    corrected_chunks = []
    for chunk in chunks:
        corrected_chunks.append(str(spell(chunk)))   
    # Combine the corrected chunks into a single string
    corrected_text = " ".join(corrected_chunks)
    return corrected_text

In [54]:
merged_parent_comments["content"] = merged_parent_comments["content"].apply(autocorrect_text)
merged_reddit_replies["content"] = merged_reddit_replies["content"].apply(autocorrect_text)

In [55]:
print(np.shape(merged_posts))
print(np.shape(merged_parent_comments))
print(np.shape(merged_reddit_replies))

(875, 8)
(42102, 5)
(34378, 6)


## <a id='toc4_5_'></a>[Lemmatization and stopword removal](#toc0_)
- The text will be converted into tokens, then these steps are performed.
- Then the text is to be converted back into one String for the transformer to interpret 

In [56]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
nltk_stopwords = set(stopwords.words('english'))

# https://gist.github.com/sebleier/554280 - source of list of stopwords
removed_stopwords = ["all", "any", "both", "each", "few", "more", "most", 
                  "other", "some", "such", "only", "own", "same", "too", 
                  "very", "s", "t", "just", "now", "not", "in"] 

for word in removed_stopwords:
        if word in nltk_stopwords:
            nltk_stopwords.remove(word)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\seanm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\seanm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\seanm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [57]:
def lemmatise_remove_stopwords(text:str) -> str:
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Lemmatize and remove stopwords
    processed_tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.lower() not in stop_words and token.isalnum()]
    # Help the model handle larger inputs
    if len(processed_tokens) > 512:
        # Truncate the tokens list to contain only the first 512 tokens
        processed_tokens = processed_tokens[:512]
    # Concatenate tokens into a string
    processed_text = ' '.join(processed_tokens)
    return processed_text

In [58]:
merged_parent_comments["content"] = merged_parent_comments["content"].apply(lemmatise_remove_stopwords)
merged_reddit_replies["content"] = merged_reddit_replies["content"].apply(lemmatise_remove_stopwords)

# <a id='toc5_'></a>[Save pre-processed data](#toc0_)

In [59]:
conn = sqlite3.connect("PreProcessed.db")

In [60]:
schema = {
    'attack_id':   'INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE',
    'attack_name': 'TEXT NOT NULL UNIQUE'
}
attacks.to_sql(name = "attacks", con = conn, if_exists = "replace", dtype = schema)

3

In [61]:
schema = {
    'rp_index': 'INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE',
    'post_id': 'TEXT NOT NULL',
    'subreddit': 'TEXT',
    'title': 'TEXT NOT NULL',
    'content': 'TEXT',
    'score': 'INTEGER DEFAULT 0',
    'upvote_ratio': 'FLOAT',
    'post_datetime': 'TIMESTAMP',
    'attack_id': 'INTEGER',
    'FOREIGN KEY (attack_id)': 'REFERENCES attacks(attack_id)'
}
merged_posts.to_sql(name = "reddit_posts", con = conn, if_exists = "replace", dtype = schema)

875

In [62]:
schema = {
    'rc_index': 'INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE',
    'parent_comment_id': 'TEXT NOT NULL',
    'content': "TEXT",
    'score': 'INTEGER DEFAULT 0',
    'comment_datetime': 'TIMESTAMP NOT NULL',
    'post_id': 'TEXT NOT NULL',
    'FOREIGN KEY (post_id)': 'REFERENCES reddit_posts(post_id)'
}
merged_parent_comments.to_sql(name = "reddit_parent_comments", if_exists = "replace", con = conn,  dtype = schema)

42102

In [63]:
schema = {
    'rr_index': 'INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE',
    'reply_id': 'TEXT NOT NULL',
    'content': 'TEXT',
    'score': 'INTEGER DEFAULT 0',
    'comment_datetime': 'TIMESTAMP NOT NULL',
    'parent_id': 'TEXT NOT NULL', 
    'post_id':   'TEXT  NOT NULL',
    'FOREIGN KEY (parent_id)': 'REFERENCES reddit_parent_comments(parent_comment_id)',
    'FOREIGN KEY (post_id)': 'REFERENCES reddit_posts(post_id)'    
}
merged_reddit_replies.to_sql(name="reddit_replies", con=conn, if_exists="replace", dtype=schema)

34378

In [64]:
conn.close()